This version is implemented based on Sage Math software

In [4]:
import logging
import numpy as np
from functools import reduce
import numpy as np
logging.basicConfig(level=logging.DEBUG)

p=random_prime(2^12)
Zp = GF(p)  # Finite Field of size 67

In [5]:
def get_random_prime_integer(bounds):
    for i in range(bounds.__len__() - 1):
        if bounds[i + 1] > bounds[i]:
            x = bounds[i] + np.random.randint(bounds[i + 1] - bounds[i])
            if is_prime(x):
                return x
        else:
            if is_prime(bounds[i]):
                return bounds[i]

        if is_prime(bounds[i + 1]):
            return bounds[i + 1]

    newBounds = [0 for i in range(2 * bounds.__len__() - 1)]
    newBounds[0] = bounds[0]
    for i in range(1, bounds.__len__()):
        newBounds[2 * i - 1] = int((bounds[i - 1] + bounds[i]) / 2)
        newBounds[2 * i] = bounds[i]

    return get_random_prime_integer(newBounds)


def is_prime(x):
    count = 0
    for i in range(int(x / 2)):
        if x % (i + 1) == 0:
            count = count + 1
    return count == 1

def generate_random_prime(min=1, max=100, nums=10):
    numbers = []
    for i in range(nums):
        numbers.append(get_random_prime_integer([min, max]))
    return np.array(numbers)

In [6]:
generate_random_prime(nums=10)

array([41, 37,  7, 43, 59, 13, 43, 19, 29, 41])

In [7]:
# to store vectorlist for mapping from alphabet to vectors
vectorlist = matrix(53)
alphabetlist='ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz '
for i in range(53):
   vectorlist[i,i] = 1

#input: alphabet
#ouput: vector
#53 symbols to vectors. a-z,A-Z,and blank.
def MapAlphabetToVector(x):
    i=alphabetlist.index(x)
    return vectorlist[i]

In [8]:
#find a number in a list
#not find, return -1
def FindinList(tem, lxs):
    lenlist=len(lxs)
    ret=-1
    for x in range(lenlist):
        if lxs[x]==tem:
            ret=x
            break
    return ret

In [9]:
#generate n randome numbers, not repeated
def GenR(n, Zp):
    l_xs=[]
    tem = Zp.random_element() # Get next primary number in FP
    while tem == 0:
        tem = Zp.random_element()
    l_xs.append(tem)
    while len(l_xs) < n:
        tem = Zp.random_element()
        while tem==0:
            tem = Zp.random_element()
        if FindinList(tem,l_xs)<0:
            l_xs.append(tem)
    return l_xs


In [24]:
#input:prime number, order
#output a efficiency list of a polynomial
def GenPolynomial(p, n_Polynomialorder):
    Zp = GF(p)
    n_efficiency = []
    for i in range(n_Polynomialorder + 1):
        n_efficiency.append(Zp.random_element())
    return n_efficiency

In [27]:
GenPolynomial(p, 3)

[9, 39, 9, 28]

In [18]:
#input: order, efficiency, xvlaues
#output: Result
def EvalPolynomial(n_Polynomialorder, n_efficiency, xvlaues):
    result=0
    for i in range(n_Polynomialorder + 1):
        result=result + n_efficiency[i] * xvlaues^i
    return result

In [19]:
# Create a secret y among n_clounds with polynomial degree n_order
def secretshare(y, n_clouds, n_order, l_xs):
    shares = []
    f_poly = GenPolynomial(p, n_order)
    f_poly[0] = y
    for x in range(n_clouds):
        resultValue = EvalPolynomial(n_order, f_poly, l_xs[x])
        shares.append(resultValue)
    return shares

In [20]:
def interpolate(x_values, y_values, x=0):
    def _basis(j):
        p = [(x - x_values[m]) / (x_values[j] - x_values[m]) for m in range(k) if m != j]
        return reduce(np.multiply, p)

    assert len(x_values) != 0 and (len(x_values) == len(y_values)), \
        "x and y cannot be empty and must have the same length"
    k = len(x_values)
    return sum(_basis(j) * y_values[j] for j in range(k))

In [21]:
logging.info('main program\n')
logging.info('----------------------------------------------------------------------')
# main program

INFO:root:main program

INFO:root:----------------------------------------------------------------------


In [22]:
# number of clouds
n_clouds=10
# genearte n_clouds x values for each cloud as an identification of an clound
l_xs=[]
l_xs=GenR(n_clouds, Zp)
logging.info(f"The finite filed we are using: Zp={Zp}")
logging.info(f"cloud numbers: {len(l_xs)}")
logging.info(f"x values for each cloud: {l_xs}")

INFO:root:The finite filed we are using: Zp=Finite Field of size 59
INFO:root:cloud numbers: 10
INFO:root:x values for each cloud: [28, 58, 47, 1, 7, 10, 49, 23, 56, 19]


In [23]:
# input one file to share
FiletoBestored='Alice Love Bob and Bob Love too'.strip()
logging.info(f"file stream to be searched: {FiletoBestored}")
# Map FiletoBestored to VectorList
vector=[]

# for temporary store vector
l_Filshares=[]
# for one file in one cloud, one row for one cloud. (for example 10 clouds)
# in each row, every cell for one alphabet (for example each cell consists of 53 elements)

for x in range(n_clouds):
    l_Filshares.append([])

INFO:root:file stream to be searched: Alice Love Bob and Bob Love too


In [18]:
shares=secretshare(1, n_clouds, 1, l_xs)  # one degree sharing, so we need 1 here.
shares

[307, 653, 162, 611, 2209, 2029, 2156, 576, 1341, 1374]

In [19]:
x = FiletoBestored[0]
vector=MapAlphabetToVector(x) # Get a vector representation for a char x
logging.info(f"The char [{x}] --> {vector}")

INFO:root:The char [A] --> (1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)


In [20]:
l_y_vectorshares=[] # for one vector,or for one symbol
for xx in range(n_clouds):
    l_y_vectorshares.append([])
l_y_vectorshares

[[], [], [], [], [], [], [], [], [], []]

In [21]:
# for xx in range(n_clouds):
#     l_y_vectorshares[xx].append(shares[xx])
# l_y_vectorshares

In [22]:
for x in FiletoBestored:
    vector=MapAlphabetToVector(x) # Get a vector representation for a char x

    #secret share each vector
    l_y_vectorshares=[] # for one vector,or for one symbol
    for xx in range(n_clouds):
        l_y_vectorshares.append([])

    shares=[]  #for one element (0 or 1)
    for y in vector:
        shares=secretshare(y, n_clouds, 1, l_xs)  # one degree sharing, so we need 1 here.
        for xx in range(n_clouds):
            l_y_vectorshares[xx].append(shares[xx])
    
    # send to clouds
    # transfer data in l_y_vectorshares to l_Filshares
    for xx in range(n_clouds):
        l_Filshares[xx].append(l_y_vectorshares[xx])


In [23]:
logging.info(f"len(l_Filshares) = {len(l_Filshares)}, len(l_Filshares[0]) = {len(l_Filshares[0])} , len(l_Filshares[0][0]) = {len(l_Filshares[0][0])}")

logging.info(f"l_Filshares[0][0]: {l_Filshares[0][0]}")

INFO:root:len(l_Filshares) = 10, len(l_Filshares[0]) = 31 , len(l_Filshares[0][0]) = 53
INFO:root:l_Filshares[0][0]: [487, 829, 665, 2363, 1263, 2293, 443, 992, 440, 1225, 1886, 1160, 2165, 46, 1443, 786, 1239, 1611, 1174, 2054, 1334, 1155, 2306, 1738, 674, 445, 658, 252, 1490, 2231, 1398, 1000, 1810, 2322, 329, 471, 2, 792, 1279, 2227, 810, 2012, 285, 508, 31, 46, 282, 933, 2297, 1875, 1478, 2002, 2095]


In [24]:
logging.info('distribute file ending...')
logging.info('-----------------------------------')

INFO:root:distribute file ending...
INFO:root:-----------------------------------


In [25]:
logging.info('Generate AA now')

n_node=5
l_Node=[]
l_NodesharesforC=[]
for i in range(n_node):
    l_Node.append(0)

INFO:root:Generate AA now


In [26]:
l_Node

[0, 0, 0, 0, 0]

In [27]:
# set initial value for each node
l_Node[0]=1  #for 'L'
l_Node[1]=0  #for 'o'
l_Node[2]=0  #for 'v'
l_Node[3]=0  #for 'e'
l_Node[4]=0

In [28]:
l_Node

[1, 0, 0, 0, 0]

In [29]:
# set positon for the word 'Love'
n_1to2=alphabetlist.index('L') #it is 11
n_2to3=alphabetlist.index('o') #it is 40
n_3to4=alphabetlist.index('v') #it is 47
n_4to5=alphabetlist.index('e') #it is 30

In [30]:
logging.info(f"Index [L -> {n_1to2}], [o -> {n_2to3}], [v -> {n_3to4}], [e -> {n_4to5}]")

INFO:root:Index [L -> 11], [o -> 40], [v -> 47], [e -> 30]


In [31]:
#set initial cell for n_clouds clouds
for xx in range(n_clouds):
    l_NodesharesforC.append([])

In [32]:
l_NodesharesforC

[[], [], [], [], [], [], [], [], [], []]

In [33]:
# secret share AA
# each share for x cloud is stored in l_NodesharesforC[x]
for i in range(n_node):
    shares = secretshare(l_Node[i], n_clouds, i + 2, l_xs)
    logging.info(f"Secret[{l_Node[i]}], Node[{i}], Shares: {shares}")
    for xx in range(n_clouds):
#         logging.info(f"Secret[{l_Node[i]}], Cloud[{xx}], Shares: {shares[xx]}")
        l_NodesharesforC[xx].append(shares[xx])

INFO:root:Secret[1], Node[0], Shares: [2126, 229, 222, 584, 782, 1157, 1559, 1179, 320, 748]
INFO:root:Secret[0], Node[1], Shares: [274, 215, 2188, 1024, 1006, 2110, 86, 308, 1228, 370]
INFO:root:Secret[0], Node[2], Shares: [690, 1736, 2167, 1743, 541, 1407, 1260, 1371, 169, 1267]
INFO:root:Secret[0], Node[3], Shares: [1951, 610, 871, 2158, 1551, 150, 1346, 757, 2267, 2129]
INFO:root:Secret[0], Node[4], Shares: [1662, 1795, 1629, 1164, 1091, 1522, 2122, 1793, 1585, 709]


In [34]:
logging.info(f"len(l_NodesharesforC) = {len(l_NodesharesforC)}, len(l_NodesharesforC[0]) = {len(l_NodesharesforC[0])}")

logging.info(f"l_NodesharesforC[0]: {l_NodesharesforC[0]}")

INFO:root:len(l_NodesharesforC) = 10, len(l_NodesharesforC[0]) = 5
INFO:root:l_NodesharesforC[0]: [2126, 274, 690, 1951, 1662]


The state of each node will be shared among 10 clounds, as shown in l_NodesharesforC. In this object, each row means five shares state of nodes in a cloud server.

In [35]:
l_NodesharesforC

[[2126, 274, 690, 1951, 1662],
 [229, 215, 1736, 610, 1795],
 [222, 2188, 2167, 871, 1629],
 [584, 1024, 1743, 2158, 1164],
 [782, 1006, 541, 1551, 1091],
 [1157, 2110, 1407, 150, 1522],
 [1559, 86, 1260, 1346, 2122],
 [1179, 308, 1371, 757, 1793],
 [320, 1228, 169, 2267, 1585],
 [748, 370, 1267, 2129, 709]]

In [36]:
logging.info(f"original AA:N0,N1,N2,N3,N4 -> {l_Node}")
logging.info(f"Generate AA ending...")
logging.info("---------------------------------------")

INFO:root:original AA:N0,N1,N2,N3,N4 -> [1, 0, 0, 0, 0]
INFO:root:Generate AA ending...
INFO:root:---------------------------------------


In [37]:
# todo upate AA in clouds.
logging.info("Updating AA now")

# each cloud will update AA for theirselves
FileLen=len(l_Filshares[0]) # it is 14 for 'Alice Love Bob'
FileLen

INFO:root:Updating AA now


31

In [38]:
for i in range(FileLen):
    # for each symbol in file stream
    for xx in range(n_clouds):
        l_NodesharesforC[xx][4] = l_NodesharesforC[xx][4] + l_NodesharesforC[xx][3] * l_Filshares[xx][i][n_4to5]
        l_NodesharesforC[xx][3] = l_NodesharesforC[xx][2] * l_Filshares[xx][i][n_3to4]
        l_NodesharesforC[xx][2] = l_NodesharesforC[xx][1] * l_Filshares[xx][i][n_2to3]
        l_NodesharesforC[xx][1] = l_NodesharesforC[xx][0] * l_Filshares[xx][i][n_1to2]

In [39]:
logging.info("The state machine is updated according to input")
l_NodesharesforC

INFO:root:The state machine is updated according to input


[[2126, 2280, 1918, 623, 1116],
 [229, 2239, 1142, 1483, 1015],
 [222, 950, 1532, 877, 109],
 [584, 440, 554, 2232, 864],
 [782, 1483, 1578, 703, 646],
 [1157, 1112, 2277, 1014, 1305],
 [1559, 117, 268, 350, 321],
 [1179, 1389, 1049, 925, 1779],
 [320, 46, 1036, 70, 2205],
 [748, 108, 296, 743, 1419]]

In [40]:
logging.info("Upate AA ending...")
logging.info("--------------------------------------------")

# todo Reconstruct AA
logging.info("Reconstruct AA now")
# From shares got from clouds
# which are stored in list l_NodesharesforC
n_node = 5
for i in range(n_node):
    l_Node[i] = 0
logging.info(f"l_Node is reinitilized {l_Node}")

INFO:root:Upate AA ending...
INFO:root:--------------------------------------------
INFO:root:Reconstruct AA now
INFO:root:l_Node is reinitilized [0, 0, 0, 0, 0]


In [41]:
logging.info(f"The identifier of each party server is: l_xs={l_xs}")
for xx in range(5):
    lxs = []
    lys = []
    for y in range(xx + 3):
        lxs.append(l_xs[y])
        lys.append(l_NodesharesforC[y][xx])
    
    l_Node[xx] = interpolate(lxs, lys)
    logging.info(f"l_Node[{xx}] = {l_Node[xx]} is recovered using lxs={lxs}, lys={lys}")


INFO:root:The identifier of each party server is: l_xs=[391, 569, 866, 2100, 2029, 1799, 1301, 1395, 1184, 830]
INFO:root:l_Node[0] = 1 is recovered using lxs=[391, 569, 866], lys=[2126, 229, 222]
INFO:root:l_Node[1] = 0 is recovered using lxs=[391, 569, 866, 2100], lys=[2280, 2239, 950, 440]
INFO:root:l_Node[2] = 0 is recovered using lxs=[391, 569, 866, 2100, 2029], lys=[1918, 1142, 1532, 554, 1578]
INFO:root:l_Node[3] = 0 is recovered using lxs=[391, 569, 866, 2100, 2029, 1799], lys=[623, 1483, 877, 2232, 703, 1014]
INFO:root:l_Node[4] = 2 is recovered using lxs=[391, 569, 866, 2100, 2029, 1799, 1301], lys=[1116, 1015, 109, 864, 646, 1305, 321]


In [42]:
# the second node

logging.info(f"Reconstruct AA:N0,N1,N2,N3,N4 -> {l_Node}")
logging.info("Reconstruct AA ending...")
logging.info("-------------------------------------")

INFO:root:Reconstruct AA:N0,N1,N2,N3,N4 -> [1, 0, 0, 0, 2]
INFO:root:Reconstruct AA ending...
INFO:root:-------------------------------------
